### データ取得テスト

In [ ]:
import json
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments

api_key = 'oanda APIのAPIKEYに置き換える'
api = API(access_token=api_key, environment="practice", headers={"Accept-Datetime-Format":"Unix"})
params = {
    'count': 3,
    'granularity': 'H1',
}
r = instruments.InstrumentsCandles(instrument='USD_JPY', params=params)
response = api.request(r)
print(json.dumps(response, indent=2))

### データ取得

In [ ]:
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import pandas as pd
from datetime import datetime

# start～endまでのデータ取得
def get_period_data(start, end, minute, instrument='USD_JPY'):
    timestamp = start.timestamp()
    concats = []
    count = 5000
    while True:
        df, last_timestamp = send_api(count, timestamp, minute, instrument)
        concats.append(df)
        if last_timestamp > end.timestamp() or len(df) < count:
            break
        timestamp = last_timestamp + (60 * minute)
    df = pd.concat(concats)
    if end is None:
        return df
    else: 
        return df[df.index < end]

# 時間足のdfを取得
def send_api(count, start, minute, instrument):
    # oandaへのリクエストの送信
    api_key = 'oanda APIのAPIKEYに置き換える'
    api = API(access_token=api_key, environment="practice", headers={"Accept-Datetime-Format":"Unix"})
    if minute == 1:
        granularity = 'M1'
    elif minute == 5:
        granularity = 'M5'
    elif minute == 15:
        granularity = 'M15'
    elif minute == 60:
        granularity = 'H1'
    params = {
        'count': count,
        'granularity': granularity,
    }
    if start is not None:
        params['from'] = start
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    response = api.request(r)
    
    # レスポンスの整形
    def join_json(candle):
        tmp = candle['mid']
        tmp['time'] = candle['time']
        tmp['v'] = candle['volume']
        tmp['complete'] = candle['complete']
        return tmp
    data_list = [join_json(candle) for candle in response['candles']]
    df = pd.DataFrame(data_list)
    last_timestamp = int(float(df.iloc[-1]['time']))
    
    # 型変更
    df['time'] = df['time'].astype('float64')
    df['o'] = df['o'].astype('float64')
    df['h'] = df['h'].astype('float64')
    df['l'] = df['l'].astype('float64')
    df['c'] = df['c'].astype('float64')
    df['v'] = df['v'].astype('float64')
    
    # タイムゾーンの変更、インデックス化
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df['time'] = df['time'] + pd.Timedelta('09:00:00') # 日本時間へ変換
    df.set_index('time', inplace=True)                 # 時間をインデックスにする
    df = df.loc[:,['o','h','l', 'c', 'v', 'complete']] # 列の順番変更
    df = df.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close', 'v': 'amount'})
    
    return df, last_timestamp

minute = 60
start = datetime.strptime('2018-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
end = datetime.strptime('2019-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
df = get_period_data(start, end, minute, instrument='USD_JPY')
print(df)

### 整形

In [ ]:
def make_learn_data(period, input_df, is_learn=False):
    df = input_df.copy()
    
    # 平均移動線
    sma5 = df['close'].rolling(5).mean()
    df['sma5_diff'] = (df['close'] - sma5) / df['close']
    sma25 = df['close'].rolling(25).mean()
    df['sma25_diff'] = (df['close'] - sma25) / df['close']
    
    # ボリンジャーバンド
    mean = df['close'].rolling(20).mean()
    std = df['close'].rolling(20).std()
    df['upper2'] = mean + (std * 2)
    df['lower2'] = lower = mean - (std * 2)
    df['upper2_diff'] = (df['upper2'] - df['close']) / df['close']
    df['lower2_diff'] = (df['lower2'] - df['close']) / df['close']
    
    # RSI
    diff = df['close'].diff()
    up, down = diff.copy(), diff.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    up_sma_14 = up.rolling(window=14, center=False).mean()
    down_sma_14 = down.abs().rolling(window=14, center=False).mean()
    RS = up_sma_14 / down_sma_14
    df['RSI'] = 100.0 - (100.0 / (1.0 + RS))
                
    # min/max_diff
    df['min'] = df['close'].rolling(window=period).min().shift(1)
    df['min_diff'] = (df['close'] - df['min']) / df['min']
    df['max'] = df['close'].rolling(window=period).max().shift(1)
    df['max_diff'] = (df['close'] - df['max']) / df['max']
            
    # ストキャスティクス
    df['L14'] = df['low'].rolling(window=14).min()
    df['H14'] = df['high'].rolling(window=14).max()
    df['%K'] = 100*((df['close'] - df['L14']) / (df['H14'] - df['L14']) )
    df['%D'] = df['%K'].rolling(window=3).mean()
    
    # openとcloseの変化率
    df['open_close_rate'] = (df['close'] - df['open']) / df['open']
    df['before_open_close_rate'] = df['open_close_rate'].shift(1)
    df['before_before_open_close_rate'] = df['open_close_rate'].shift(2)

    # ローソク足
    df.loc[df['close'] - df['open'] >= 0, 'candle'] = 1
    df.loc[df['close'] - df['open'] < 0, 'candle'] = 0
    df.loc[df['candle'] == 1, 'high_rate'] = (df['high'] - df['close']) / df['close']
    df.loc[df['candle'] == 0, 'high_rate'] = (df['high'] - df['open']) / df['open']
    df.loc[df['candle'] == 1, 'low_rate'] = (df['open'] - df['low']) / df['open']
    df.loc[df['candle'] == 0, 'low_rate'] = (df['close'] - df['low']) / df['close']
    df.loc[df['candle'] == 1, 'amount2'] = df['amount']
    df.loc[df['candle'] == 0, 'amount2'] = -df['amount']
    
    # n時間前からの変化率
    for i in range(period):
        df['change_rate{}'.format(i+1)] = (df['close'] - df['close'].shift(i+1)) / df['close'].shift(i+1)

    # before/after_rate
    rate = 0
    df['before_rate'] = (df['close'] - df['close'].shift(1)) / df['close'].shift(1)
    if is_learn:
        df['after_rate'] = (df['close'].shift(-1) - df['open'].shift(-1)) / df['open'].shift(-1)
        df['updown'] = 0
        df.loc[df['after_rate'] > rate, 'updown'] = 1
        df.loc[df['after_rate'] <= -rate, 'updown'] = -1

    df = df.dropna()   
    return df

newdf = make_learn_data(24, df, True)
print(newdf)

In [ ]:
from sklearn import preprocessing
import pickle

def makeXY(df, filename=None, sc=None, is_sc=True, is_learn=False):
    explain_val = df.columns.values.tolist()
    delete_columns = [
        'candle', 'complete', 'after_rate', 'updown', 
        'upper2', 'lower2', 'min', 'max', 'L14', 'H14']
    for column in delete_columns:
        if column in df.columns:
            explain_val.remove(column)
    X = df.loc[:, explain_val]

    if is_learn:
        response_val = 'updown'
        Y = df.loc[:, response_val]
    if sc is None:
        sc = preprocessing.StandardScaler()
        sc.fit(X)
        if is_sc:
            pickle.dump(sc, open(filename, 'wb'))
    X = sc.transform(X)
    
    if is_learn:
        return X, Y, explain_val
    return X, df.index[-1]

X, Y, explain_val = makeXY(newdf, 'sc.sav', None, True, True)
print('X={}\nY={}\nexplain_val={}'.format(X, Y, explain_val))

### 学習

In [ ]:
import xgboost as xgb
import plotly.offline as offline
import plotly.graph_objs as go
offline.init_notebook_mode(connected=False)

test_period = 500
X_train, Y_train, X_test, Y_test = X[:-test_period], Y[:-test_period], X[-test_period:], Y[-test_period:]

clf = xgb.XGBClassifier()
clf.fit(X_train, Y_train)
pickle.dump(clf, open('xgb.sav', 'wb'))
data = [go.Bar(x=explain_val,y=clf.feature_importances_)]
offline.iplot(data)

In [ ]:
def calc_benefit(clf, X_test, Y_test, df_test):
    diff = (df_test['close'].shift(-1) - df_test['open'].shift(-1)) * 100 / df_test['open'].shift(-1)
    
    Y_pred = clf.predict(X_test)
    Y_real = Y_test.copy()
    print('テストに使ったデータ {}～{}'.format(Y_real.index[0], Y_real.index[-1]))

    benefit = 0
    history = pd.DataFrame([], columns = ['benefit'])
    
    right_count = 0
    all_count = 0

    for i in range(len(Y_real)):
        # 最終行は次がないので終わり
        if i == len(Y_real) - 1:
            break
            
        if Y_pred[i] == 1:
            all_count += 1
            benefit += diff[i]
            history.loc[Y_real.index[i+1]] = benefit
            if Y_real[i] == 1:
                right_count += 1
        elif Y_pred[i] == -1:
            all_count += 1
            benefit -= diff[i]
            history.loc[Y_real.index[i+1]] = benefit
            if Y_real[i] == -1:
                right_count += 1
                            
    data = []
    candlestick = go.Candlestick(x=df_test.index,
                       open=df_test.open,
                       high=df_test.high,
                       low=df_test.low,
                       close=df_test.close)
    data.append(candlestick)
    data.append(go.Scatter(x=history.index, y=history['benefit'], mode='markers+lines', yaxis='y2'))
    layout = go.Layout(
        xaxis = dict(fixedrange=True, rangeslider=dict(visible=False)),
        yaxis = dict(side='left', showgrid=False),
        yaxis2 = dict(side='right', overlaying='y', showgrid=False)
    )
    offline.iplot({'data':data,'layout':layout})

    if all_count == 0:
        print('No Trades')
    else:
        print('all: {}, right: {}, accuracy: {}%'.format(all_count, right_count, round(right_count * 100 / all_count, 1)))
    print('benefit = {}%'.format(round(benefit, 3)))
    return benefit

In [ ]:
df_test = df[-test_period:]
calc_benefit(clf, X_test, Y_test, df_test)

### 予測偏

In [ ]:
def predict(sc, clf, newdf):
    X, X_last_date = makeXY(newdf, sc=sc, is_sc=False, is_learn=False)
    Y_pred = clf.predict(X)
    return Y_pred[-1]

count = 30
period = 24

# sc, xgb取得
sc = pickle.load(open('sc.sav'.format(minute), 'rb'))
clf = pickle.load(open('xgb.sav'.format(minute), 'rb'))

# リアルタイムデータ取得・整形
df, last_timestamp = send_api(count, None, minute, 'USD_JPY')
newdf = make_learn_data(period, df, is_learn=False)

# 予測
print(newdf)
Y = predict(sc, clf, newdf)
if Y == 1:
    # 買う処理
    print('buy')
elif Y == -1:
    # 売る処理
    print('sell')

### 忙しい人のため

#### 学習＆テスト

In [ ]:
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments

import pickle
from datetime import datetime

import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
import plotly.offline as offline
import plotly.graph_objs as go
offline.init_notebook_mode(connected=False)

# start～endまでのデータ取得
def get_period_data(start, end, minute, instrument='USD_JPY'):
    timestamp = start.timestamp()
    concats = []
    count = 5000
    while True:
        df, last_timestamp = send_api(count, timestamp, minute, instrument)
        concats.append(df)
        if last_timestamp > end.timestamp() or len(df) < count:
            break
        timestamp = last_timestamp + (60 * minute)
    df = pd.concat(concats)
    if end is None:
        return df
    else: 
        return df[df.index < end]

# 時間足のdfを取得
def send_api(count, start, minute, instrument):
    # oandaへのリクエストの送信
    api_key = 'oanda APIのAPIKEYに置き換える'
    api = API(access_token=api_key, environment="practice", headers={"Accept-Datetime-Format":"Unix"})
    if minute == 1:
        granularity = 'M1'
    elif minute == 5:
        granularity = 'M5'
    elif minute == 15:
        granularity = 'M15'
    elif minute == 60:
        granularity = 'H1'
    params = {
        'count': count,
        'granularity': granularity,
    }
    if start is not None:
        params['from'] = start
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    response = api.request(r)
    
    # レスポンスの整形
    def join_json(candle):
        tmp = candle['mid']
        tmp['time'] = candle['time']
        tmp['v'] = candle['volume']
        tmp['complete'] = candle['complete']
        return tmp
    data_list = [join_json(candle) for candle in response['candles']]
    df = pd.DataFrame(data_list)
    last_timestamp = int(float(df.iloc[-1]['time']))
    
    # 型変更
    df['time'] = df['time'].astype('float64')
    df['o'] = df['o'].astype('float64')
    df['h'] = df['h'].astype('float64')
    df['l'] = df['l'].astype('float64')
    df['c'] = df['c'].astype('float64')
    df['v'] = df['v'].astype('float64')
    
    # タイムゾーンの変更、インデックス化
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df['time'] = df['time'] + pd.Timedelta('09:00:00') # 日本時間へ変換
    df.set_index('time', inplace=True)                 # 時間をインデックスにする
    df = df.loc[:,['o','h','l', 'c', 'v', 'complete']] # 列の順番変更
    df = df.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close', 'v': 'amount'})
    
    return df, last_timestamp

def make_learn_data(period, input_df, is_learn=False):
    df = input_df.copy()
    
    # 平均移動線
    sma5 = df['close'].rolling(5).mean()
    df['sma5_diff'] = (df['close'] - sma5) / df['close']
    sma25 = df['close'].rolling(25).mean()
    df['sma25_diff'] = (df['close'] - sma25) / df['close']
    
    # ボリンジャーバンド
    mean = df['close'].rolling(20).mean()
    std = df['close'].rolling(20).std()
    df['upper2'] = mean + (std * 2)
    df['lower2'] = lower = mean - (std * 2)
    df['upper2_diff'] = (df['upper2'] - df['close']) / df['close']
    df['lower2_diff'] = (df['lower2'] - df['close']) / df['close']
    
    # RSI
    diff = df['close'].diff()
    up, down = diff.copy(), diff.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    up_sma_14 = up.rolling(window=14, center=False).mean()
    down_sma_14 = down.abs().rolling(window=14, center=False).mean()
    RS = up_sma_14 / down_sma_14
    df['RSI'] = 100.0 - (100.0 / (1.0 + RS))
                
    # min/max_diff
    df['min'] = df['close'].rolling(window=period).min().shift(1)
    df['min_diff'] = (df['close'] - df['min']) / df['min']
    df['max'] = df['close'].rolling(window=period).max().shift(1)
    df['max_diff'] = (df['close'] - df['max']) / df['max']
            
    # ストキャスティクス
    df['L14'] = df['low'].rolling(window=14).min()
    df['H14'] = df['high'].rolling(window=14).max()
    df['%K'] = 100*((df['close'] - df['L14']) / (df['H14'] - df['L14']) )
    df['%D'] = df['%K'].rolling(window=3).mean()
    
    # openとcloseの変化率
    df['open_close_rate'] = (df['close'] - df['open']) / df['open']
    df['before_open_close_rate'] = df['open_close_rate'].shift(1)
    df['before_before_open_close_rate'] = df['open_close_rate'].shift(2)

    # ローソク足
    df.loc[df['close'] - df['open'] >= 0, 'candle'] = 1
    df.loc[df['close'] - df['open'] < 0, 'candle'] = 0
    df.loc[df['candle'] == 1, 'high_rate'] = (df['high'] - df['close']) / df['close']
    df.loc[df['candle'] == 0, 'high_rate'] = (df['high'] - df['open']) / df['open']
    df.loc[df['candle'] == 1, 'low_rate'] = (df['open'] - df['low']) / df['open']
    df.loc[df['candle'] == 0, 'low_rate'] = (df['close'] - df['low']) / df['close']
    df.loc[df['candle'] == 1, 'amount2'] = df['amount']
    df.loc[df['candle'] == 0, 'amount2'] = -df['amount']
    
    # n時間前からの変化率
    for i in range(period):
        df['change_rate{}'.format(i+1)] = (df['close'] - df['close'].shift(i+1)) / df['close'].shift(i+1)

    # before/after_rate
    rate = 0
    df['before_rate'] = (df['close'] - df['close'].shift(1)) / df['close'].shift(1)
    if is_learn:
        df['after_rate'] = (df['close'].shift(-1) - df['open'].shift(-1)) / df['open'].shift(-1)
        df['updown'] = 0
        df.loc[df['after_rate'] > rate, 'updown'] = 1
        df.loc[df['after_rate'] <= -rate, 'updown'] = -1

    df = df.dropna()   
    return df

def makeXY(df, filename=None, sc=None, is_sc=True, is_learn=False):
    explain_val = df.columns.values.tolist()
    delete_columns = [
        'candle', 'complete', 'after_rate', 'updown', 
        'upper2', 'lower2', 'min', 'max', 'L14', 'H14']
    for column in delete_columns:
        if column in df.columns:
            explain_val.remove(column)
    X = df.loc[:, explain_val]

    if is_learn:
        response_val = 'updown'
        Y = df.loc[:, response_val]
    if sc is None:
        sc = preprocessing.StandardScaler()
        sc.fit(X)
        if is_sc:
            pickle.dump(sc, open(filename, 'wb'))
    X = sc.transform(X)
    
    if is_learn:
        return X, Y, explain_val
    return X, df.index[-1]

def calc_benefit(clf, X_test, Y_test, df_test):
    diff = (df_test['close'].shift(-1) - df_test['open'].shift(-1)) * 100 / df_test['open'].shift(-1)
    
    Y_pred = clf.predict(X_test)
    Y_real = Y_test.copy()
    print('テストに使ったデータ {}～{}'.format(Y_real.index[0], Y_real.index[-1]))

    benefit = 0
    history = pd.DataFrame([], columns = ['benefit'])
    
    right_count = 0
    all_count = 0

    for i in range(len(Y_real)):
        # 最終行は次がないので終わり
        if i == len(Y_real) - 1:
            break
            
        if Y_pred[i] == 1:
            all_count += 1
            benefit += diff[i]
            history.loc[Y_real.index[i+1]] = benefit
            if Y_real[i] == 1:
                right_count += 1
        elif Y_pred[i] == -1:
            all_count += 1
            benefit -= diff[i]
            history.loc[Y_real.index[i+1]] = benefit
            if Y_real[i] == -1:
                right_count += 1
                            
    data = []
    candlestick = go.Candlestick(x=df_test.index,
                       open=df_test.open,
                       high=df_test.high,
                       low=df_test.low,
                       close=df_test.close)
    data.append(candlestick)
    data.append(go.Scatter(x=history.index, y=history['benefit'], mode='markers+lines', yaxis='y2'))
    layout = go.Layout(
        xaxis = dict(fixedrange=True, rangeslider=dict(visible=False)),
        yaxis = dict(side='left', showgrid=False),
        yaxis2 = dict(side='right', overlaying='y', showgrid=False)
    )
    offline.iplot({'data':data,'layout':layout})

    if all_count == 0:
        print('No Trades')
    else:
        print('all: {}, right: {}, accuracy: {}%'.format(all_count, right_count, round(right_count * 100 / all_count, 1)))
    print('benefit = {}%'.format(round(benefit, 3)))
    return benefit

# --- main --- #

# データ取得
minute = 60
start = datetime.strptime('2018-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
end = datetime.strptime('2019-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
df = get_period_data(start, end, minute, instrument='USD_JPY')

# 説明変数作成
newdf = make_learn_data(24, df, True)

# 標準化
X, Y, explain_val = makeXY(newdf, 'sc.sav', None, True, True)

# 学習モデル作成
test_period = 500
X_train, Y_train, X_test, Y_test = X[:-test_period], Y[:-test_period], X[-test_period:], Y[-test_period:]
clf = xgb.XGBClassifier()
clf.fit(X_train, Y_train)
pickle.dump(clf, open('xgb.sav', 'wb'))
data = [go.Bar(x=explain_val,y=clf.feature_importances_)]
offline.iplot(data)

# テスト
df_test = df[-test_period:]
calc_benefit(clf, X_test, Y_test, df_test)

#### 予測

In [2]:
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments

import pickle
from datetime import datetime

import pandas as pd
from sklearn import preprocessing
import xgboost as xgb

# 時間足のdfを取得
def send_api(count, start, minute, instrument):
    # oandaへのリクエストの送信
    api_key = 'oanda APIのAPIKEYに置き換える'
    api = API(access_token=api_key, environment="practice", headers={"Accept-Datetime-Format":"Unix"})
    if minute == 1:
        granularity = 'M1'
    elif minute == 5:
        granularity = 'M5'
    elif minute == 15:
        granularity = 'M15'
    elif minute == 60:
        granularity = 'H1'
    params = {
        'count': count,
        'granularity': granularity,
    }
    if start is not None:
        params['from'] = start
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    response = api.request(r)
    
    # レスポンスの整形
    def join_json(candle):
        tmp = candle['mid']
        tmp['time'] = candle['time']
        tmp['v'] = candle['volume']
        tmp['complete'] = candle['complete']
        return tmp
    data_list = [join_json(candle) for candle in response['candles']]
    df = pd.DataFrame(data_list)
    last_timestamp = int(float(df.iloc[-1]['time']))
    
    # 型変更
    df['time'] = df['time'].astype('float64')
    df['o'] = df['o'].astype('float64')
    df['h'] = df['h'].astype('float64')
    df['l'] = df['l'].astype('float64')
    df['c'] = df['c'].astype('float64')
    df['v'] = df['v'].astype('float64')
    
    # タイムゾーンの変更、インデックス化
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df['time'] = df['time'] + pd.Timedelta('09:00:00') # 日本時間へ変換
    df.set_index('time', inplace=True)                 # 時間をインデックスにする
    df = df.loc[:,['o','h','l', 'c', 'v', 'complete']] # 列の順番変更
    df = df.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close', 'v': 'amount'})
    
    return df, last_timestamp

def make_learn_data(period, input_df, is_learn=False):
    df = input_df.copy()
    
    # 平均移動線
    sma5 = df['close'].rolling(5).mean()
    df['sma5_diff'] = (df['close'] - sma5) / df['close']
    sma25 = df['close'].rolling(25).mean()
    df['sma25_diff'] = (df['close'] - sma25) / df['close']
    
    # ボリンジャーバンド
    mean = df['close'].rolling(20).mean()
    std = df['close'].rolling(20).std()
    df['upper2'] = mean + (std * 2)
    df['lower2'] = lower = mean - (std * 2)
    df['upper2_diff'] = (df['upper2'] - df['close']) / df['close']
    df['lower2_diff'] = (df['lower2'] - df['close']) / df['close']
    
    # RSI
    diff = df['close'].diff()
    up, down = diff.copy(), diff.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    up_sma_14 = up.rolling(window=14, center=False).mean()
    down_sma_14 = down.abs().rolling(window=14, center=False).mean()
    RS = up_sma_14 / down_sma_14
    df['RSI'] = 100.0 - (100.0 / (1.0 + RS))
                
    # min/max_diff
    df['min'] = df['close'].rolling(window=period).min().shift(1)
    df['min_diff'] = (df['close'] - df['min']) / df['min']
    df['max'] = df['close'].rolling(window=period).max().shift(1)
    df['max_diff'] = (df['close'] - df['max']) / df['max']
            
    # ストキャスティクス
    df['L14'] = df['low'].rolling(window=14).min()
    df['H14'] = df['high'].rolling(window=14).max()
    df['%K'] = 100*((df['close'] - df['L14']) / (df['H14'] - df['L14']) )
    df['%D'] = df['%K'].rolling(window=3).mean()
    
    # openとcloseの変化率
    df['open_close_rate'] = (df['close'] - df['open']) / df['open']
    df['before_open_close_rate'] = df['open_close_rate'].shift(1)
    df['before_before_open_close_rate'] = df['open_close_rate'].shift(2)

    # ローソク足
    df.loc[df['close'] - df['open'] >= 0, 'candle'] = 1
    df.loc[df['close'] - df['open'] < 0, 'candle'] = 0
    df.loc[df['candle'] == 1, 'high_rate'] = (df['high'] - df['close']) / df['close']
    df.loc[df['candle'] == 0, 'high_rate'] = (df['high'] - df['open']) / df['open']
    df.loc[df['candle'] == 1, 'low_rate'] = (df['open'] - df['low']) / df['open']
    df.loc[df['candle'] == 0, 'low_rate'] = (df['close'] - df['low']) / df['close']
    df.loc[df['candle'] == 1, 'amount2'] = df['amount']
    df.loc[df['candle'] == 0, 'amount2'] = -df['amount']
    
    # n時間前からの変化率
    for i in range(period):
        df['change_rate{}'.format(i+1)] = (df['close'] - df['close'].shift(i+1)) / df['close'].shift(i+1)

    # before/after_rate
    rate = 0
    df['before_rate'] = (df['close'] - df['close'].shift(1)) / df['close'].shift(1)
    if is_learn:
        df['after_rate'] = (df['close'].shift(-1) - df['open'].shift(-1)) / df['open'].shift(-1)
        df['updown'] = 0
        df.loc[df['after_rate'] > rate, 'updown'] = 1
        df.loc[df['after_rate'] <= -rate, 'updown'] = -1

    df = df.dropna()   
    return df

def makeXY(df, filename=None, sc=None, is_sc=True, is_learn=False):
    explain_val = df.columns.values.tolist()
    delete_columns = [
        'candle', 'complete', 'after_rate', 'updown', 
        'upper2', 'lower2', 'min', 'max', 'L14', 'H14']
    for column in delete_columns:
        if column in df.columns:
            explain_val.remove(column)
    X = df.loc[:, explain_val]

    if is_learn:
        response_val = 'updown'
        Y = df.loc[:, response_val]
    if sc is None:
        sc = preprocessing.StandardScaler()
        sc.fit(X)
        if is_sc:
            pickle.dump(sc, open(filename, 'wb'))
    X = sc.transform(X)
    
    if is_learn:
        return X, Y, explain_val
    return X, df.index[-1]

def predict(sc, clf, newdf):
    X, X_last_date = makeXY(newdf, sc=sc, is_sc=False, is_learn=False)
    Y_pred = clf.predict(X)
    return Y_pred[-1]

count = 30
period = 24
minute = 60

# sc, xgb取得
sc = pickle.load(open('sc.sav', 'rb'))
clf = pickle.load(open('xgb.sav', 'rb'))

# リアルタイムデータ取得・整形
df, last_timestamp = send_api(count, None, minute, 'USD_JPY')
newdf = make_learn_data(period, df, is_learn=False)

# 予測
Y = predict(sc, clf, newdf)
if Y == 1:
    # 買う処理
    print('buy')
elif Y == -1:
    # 売る処理
    print('sell')

sell
